# MalthusJAX Level 1 Demo: Genomes and Fitness Functions

Welcome to the **MalthusJAX Level 1 Demo**! This notebook demonstrates the core components of the MalthusJAX evolutionary computation framework:

## 🧬 What You'll Learn
- **Genome Types**: Binary, Real, Categorical, and Linear genomes
- **Fitness Functions**: BinarySum, Knapsack, Sphere, Griewank, and Box constraints  
- **Population Operations**: Creation, evaluation, and manipulation
- **JAX Integration**: High-performance JIT compilation and vectorization
- **Distance Metrics**: Genome similarity and diversity measures

## 🏗️ Level 1 Architecture Overview
Level 1 provides the **fundamental building blocks** for evolutionary computation:
- **BaseGenome**: Abstract interface for all genome types
- **BasePopulation**: Population management with vectorized operations  
- **Fitness Evaluators**: Pure JAX functions for efficient evaluation
- **Auto-correction**: Built-in validation and repair mechanisms

Let's explore these components step by step! 🚀

In [36]:
# Import Required Libraries
import jax
import jax.numpy as jnp
import jax.random as jr
import numpy as np
import matplotlib.pyplot as plt
import time

# Import MalthusJAX components
import malthusjax as mjx

print("🧪 MalthusJAX Level 1 Demo")
print(f"JAX version: {jax.__version__}")
print(f"MalthusJAX version: {mjx.__version__}")
print(f"JAX backend: {jax.default_backend()}")

# Set up random key for reproducible results
key = jr.PRNGKey(42)
print(f"\n✓ Random key initialized: {key}")

🧪 MalthusJAX Level 1 Demo
JAX version: 0.8.0
MalthusJAX version: 0.2.0
JAX backend: cpu

✓ Random key initialized: [ 0 42]


## 🔴 Binary Genomes: Combinatorial Optimization

Binary genomes represent solutions as **bit strings** - perfect for combinatorial problems like the Traveling Salesman Problem, feature selection, or the classic OneMax benchmark.

### Key Features:
- **Efficient bit operations**: `flip_bit()`, `count_ones()`, `to_int()`
- **Population vectorization**: Batch operations on thousands of genomes
- **Distance metrics**: Hamming distance for similarity measurement
- **Auto-correction**: Invalid bits automatically clamped to 0 or 1

In [37]:
# Create Binary Genomes
print("🔴 Binary Genome Demonstration")
print("="*50)

# Configuration for 20-bit binary genomes
binary_config = mjx.BinaryGenomeConfig(length=20)
key, subkey = jr.split(key)

# Create individual genomes
genome1 = mjx.BinaryGenome.random_init(subkey, binary_config)
print(f"✓ Random binary genome: {genome1}")
print(f"  - Length: {len(genome1.bits)}")
print(f"  - Number of ones: {genome1.count_ones()}")
print(f"  - Integer value: {genome1.to_int()}")
#print(f"  - Magnitude: {genome1.magnitude():.3f}")

# Create a specific pattern
specific_bits = jnp.array([1, 0, 1, 1, 0, 1, 0, 0, 1, 1, 1, 0, 0, 1, 0, 1, 1, 1, 0, 0])
genome2 = mjx.BinaryGenome(bits=specific_bits)
print(f"\n✓ Custom binary genome: {genome2}")

# Demonstrate bit operations
flipped = genome1.flip_bit(5)
print(f"\n🔧 Bit operations:")
print(f"  - Original bit[5]: {genome1.bits[5]}")
print(f"  - After flip_bit(5): {flipped.bits[5]}")

# Distance calculation
distance = genome1.distance(genome2, "hamming")
print(f"  - Hamming distance: {distance}")

print("\n" + "="*50)

🔴 Binary Genome Demonstration
✓ Random binary genome: <BinaryGenome(0011111111..., len=20)>
  - Length: 20
  - Number of ones: 14
  - Integer value: 888828

✓ Custom binary genome: <BinaryGenome(1011010011..., len=20)>

🔧 Bit operations:
  - Original bit[5]: 1
  - After flip_bit(5): 0
  - Hamming distance: 9.0



In [38]:
# Binary Populations and Fitness Functions
print("📊 Binary Fitness Functions")
print("="*50)

key, subkey = jr.split(key)

# Create a population of binary genomes
population = mjx.BinaryPopulation.init_random(subkey, binary_config, size=100)
print(f"✓ Created population: {len(population)} binary genomes")
print(f"  - Genome shape: {population.genes.bits.shape}")

# OneMax (BinarySum) Problem
print(f"\n🎯 OneMax (BinarySum) Problem:")
onemax_evaluator = mjx.BinarySumEvaluator(mjx.BinarySumConfig(maximize=True))

# Evaluate population
fitness_scores = onemax_evaluator.evaluate_batch(population)
best_fitness = max(fitness_scores)
average_fitness = sum(fitness_scores) / len(fitness_scores)

print(f"  - Best fitness: {best_fitness}/20 ones")
print(f"  - Average fitness: {average_fitness:.2f}")
print(f"  - Fitness range: [{min(fitness_scores)}, {max(fitness_scores)}]")

# Find the best genome
best_idx = fitness_scores.index(best_fitness)
best_genome = population[best_idx]
print(f"  - Best genome: {best_genome}")

print("\n" + "="*50)

📊 Binary Fitness Functions
✓ Created population: 100 binary genomes
  - Genome shape: (100, 20)

🎯 OneMax (BinarySum) Problem:
  - Best fitness: 16.0/20 ones
  - Average fitness: 9.88
  - Fitness range: [4.0, 16.0]
  - Best genome: <BinaryGenome(1010111111..., len=20)>



In [39]:
# Knapsack Problem
print("🎒 Knapsack Optimization Problem")
print("="*50)

key, subkey = jr.split(key)

# Create a random knapsack problem
n_items = 15
knapsack_config = mjx.KnapsackEvaluator.create_random_problem(subkey, n_items, capacity_ratio=0.6)
knapsack_evaluator = mjx.KnapsackEvaluator(knapsack_config)

print(f"✓ Knapsack problem created:")
print(f"  - Items: {n_items}")
print(f"  - Capacity: {knapsack_config.capacity:.1f}")
print(f"  - Total weight: {jnp.sum(knapsack_config.weights):.1f}")
print(f"  - Total value: {jnp.sum(knapsack_config.values):.1f}")

# Create population for knapsack problem  
knapsack_genome_config = mjx.BinaryGenomeConfig(length=n_items)
key, subkey = jr.split(key)
knapsack_population = mjx.BinaryPopulation.init_random(subkey, knapsack_genome_config, size=50)

# Evaluate knapsack solutions
knapsack_fitness = knapsack_evaluator.evaluate_batch(knapsack_population)
best_knapsack_fitness = max(knapsack_fitness)
best_knapsack_idx = knapsack_fitness.index(best_knapsack_fitness)
best_knapsack_genome = knapsack_population[best_knapsack_idx]

print(f"\n🏆 Best knapsack solution:")
print(f"  - Items selected: {best_knapsack_genome.count_ones()}/{n_items}")
print(f"  - Fitness (value): {best_knapsack_fitness:.1f}")
print(f"  - Solution: {best_knapsack_genome}")

# Calculate actual weight and value
selected_weights = best_knapsack_genome.bits * knapsack_config.weights
selected_values = best_knapsack_genome.bits * knapsack_config.values
total_weight = jnp.sum(selected_weights)
total_value = jnp.sum(selected_values)

print(f"  - Actual weight: {total_weight:.1f} (capacity: {knapsack_config.capacity:.1f})")
print(f"  - Actual value: {total_value:.1f}")
print(f"  - Feasible: {total_weight <= knapsack_config.capacity}")

print("\n" + "="*50)

🎒 Knapsack Optimization Problem
✓ Knapsack problem created:
  - Items: 15
  - Capacity: 93.4
  - Total weight: 155.6
  - Total value: 379.8

🏆 Best knapsack solution:
  - Items selected: 7/15
  - Fitness (value): 235.1
  - Solution: <BinaryGenome(1100010100..., len=15)>
  - Actual weight: 88.1 (capacity: 93.4)
  - Actual value: 235.1
  - Feasible: True


🏆 Best knapsack solution:
  - Items selected: 7/15
  - Fitness (value): 235.1
  - Solution: <BinaryGenome(1100010100..., len=15)>
  - Actual weight: 88.1 (capacity: 93.4)
  - Actual value: 235.1
  - Feasible: True



## 🔵 Real Genomes: Continuous Optimization

Real genomes represent solutions as **floating-point vectors** - ideal for function optimization, parameter tuning, and continuous search spaces.

### Key Features:
- **Bounded optimization**: Automatic constraint enforcement
- **Numerical operations**: `normalize()`, `add_noise()`, `magnitude()`
- **Multiple distance metrics**: Euclidean, Manhattan, and Hamming
- **Auto-correction**: Out-of-bounds values automatically clamped

In [40]:
# Create Real Genomes
print("🔵 Real Genome Demonstration")
print("="*50)

# Configuration for 8-dimensional real genomes
real_config = mjx.RealGenomeConfig(length=8, bounds=(-5.0, 5.0))
key, subkey = jr.split(key)

# Create individual genomes
real_genome1 = mjx.RealGenome.random_init(subkey, real_config)
print(f"✓ Random real genome: {real_genome1}")
print(f"  - Length: {len(real_genome1.values)}")
print(f"  - Magnitude: {real_genome1.magnitude():.3f}")
print(f"  - Bounds: {real_config.bounds}")

# Create a specific pattern
key, subkey = jr.split(key)
specific_values = jr.normal(subkey, (8,)) * 2.0
real_genome2 = mjx.RealGenome(values=specific_values)
corrected = real_genome2.autocorrect(real_config)
print(f"\n✓ Custom real genome: {real_genome2}")
print(f"  - After bounds correction: {corrected}")

# Demonstrate real operations
normalized = real_genome1.normalize()
key, subkey = jr.split(key)
noisy = real_genome1.add_noise(subkey, noise_std=0.1)

print(f"\n🔧 Real operations:")
print(f"  - Original magnitude: {real_genome1.magnitude():.3f}")
print(f"  - Normalized magnitude: {normalized.magnitude():.3f}")
print(f"  - With noise added: {noisy}")

# Distance calculations
euclidean_dist = real_genome1.distance(real_genome2, "euclidean")
manhattan_dist = real_genome1.distance(real_genome2, "manhattan")

print(f"  - Euclidean distance: {euclidean_dist:.3f}")
print(f"  - Manhattan distance: {manhattan_dist:.3f}")

print("\n" + "="*50)

🔵 Real Genome Demonstration
✓ Random real genome: <RealGenome([3.884, -2.880, 1.514, ..., -2.040], len=8)>
  - Length: 8
  - Magnitude: 7.021
  - Bounds: (-5.0, 5.0)

✓ Custom real genome: <RealGenome([1.633, -2.083, -0.188, ..., 3.387], len=8)>
  - After bounds correction: <RealGenome([1.633, -2.083, -0.188, ..., 3.387], len=8)>

🔧 Real operations:
  - Original magnitude: 7.021
  - Normalized magnitude: 1.000
  - With noise added: <RealGenome([3.781, -2.911, 1.717, ..., -1.932], len=8)>
  - Euclidean distance: 10.002
  - Manhattan distance: 23.096

  - Magnitude: 7.021
  - Bounds: (-5.0, 5.0)

✓ Custom real genome: <RealGenome([1.633, -2.083, -0.188, ..., 3.387], len=8)>
  - After bounds correction: <RealGenome([1.633, -2.083, -0.188, ..., 3.387], len=8)>

🔧 Real operations:
  - Original magnitude: 7.021
  - Normalized magnitude: 1.000
  - With noise added: <RealGenome([3.781, -2.911, 1.717, ..., -1.932], len=8)>
  - Euclidean distance: 10.002
  - Manhattan distance: 23.096



In [41]:
# Real Fitness Functions
print("📈 Real Genome Fitness Functions")
print("="*50)

# Create population for real optimization
key, subkey = jr.split(key)
real_population = mjx.RealPopulation.init_random(subkey, real_config, size=80)
print(f"✓ Created real population: {len(real_population)} genomes")

# Sphere Function (Simple Optimization)
print(f"\n🌐 Sphere Function: f(x) = sum(x_i^2)")
sphere_evaluator = mjx.SphereEvaluator(mjx.SphereConfig(minimize=True))

sphere_fitness = sphere_evaluator.evaluate_batch(real_population)
best_sphere = max(sphere_fitness)  # Max because we use negative values for minimization
best_sphere_idx = sphere_fitness.index(best_sphere)
best_sphere_genome = real_population[best_sphere_idx]

print(f"  - Best fitness: {best_sphere:.3f} (f = {-best_sphere:.3f})")
print(f"  - Best genome magnitude: {best_sphere_genome.magnitude():.3f}")
print(f"  - Global optimum: f(0,0,...,0) = 0")

# Griewank Function (Multimodal)
print(f"\n🌊 Griewank Function (Multimodal)")
# Use wider bounds for Griewank (typically [-600, 600])
griewank_config = mjx.RealGenomeConfig(length=6, bounds=(-100.0, 100.0))
key, subkey = jr.split(key)
griewank_population = mjx.RealPopulation.init_random(subkey, griewank_config, size=60)

griewank_evaluator = mjx.GriewankEvaluator(mjx.GriewankConfig(minimize=True))
griewank_fitness = griewank_evaluator.evaluate_batch(griewank_population)
best_griewank = max(griewank_fitness)

print(f"  - Best fitness: {best_griewank:.3f} (f = {-best_griewank:.3f})")
print(f"  - Fitness range: [{min(griewank_fitness):.1f}, {max(griewank_fitness):.1f}]")
print(f"  - Global optimum: f(0,0,...,0) = 0")

print("\n" + "="*50)

📈 Real Genome Fitness Functions
✓ Created real population: 80 genomes

🌐 Sphere Function: f(x) = sum(x_i^2)
  - Best fitness: -17.833 (f = 17.833)
  - Best genome magnitude: 4.223
  - Global optimum: f(0,0,...,0) = 0

🌊 Griewank Function (Multimodal)
  - Best fitness: -2.927 (f = 2.927)
  - Fitness range: [-10.8, -2.9]
  - Global optimum: f(0,0,...,0) = 0

✓ Created real population: 80 genomes

🌐 Sphere Function: f(x) = sum(x_i^2)
  - Best fitness: -17.833 (f = 17.833)
  - Best genome magnitude: 4.223
  - Global optimum: f(0,0,...,0) = 0

🌊 Griewank Function (Multimodal)
  - Best fitness: -2.927 (f = 2.927)
  - Fitness range: [-10.8, -2.9]
  - Global optimum: f(0,0,...,0) = 0



## 🟡 Categorical Genomes: Discrete Choice Optimization

Categorical genomes represent solutions as **discrete choice sequences** - perfect for problems with categorical variables, routing optimization, or permutation problems like TSP.

### Key Features:
- **Multi-category support**: Each position can take one of N discrete values
- **Permutation detection**: Built-in checking for valid permutations
- **Category operations**: `swap_positions()`, `count_category()`, `get_category_distribution()`
- **TSP conversion**: Direct conversion to traveling salesman problem format

In [42]:
# Create Categorical Genomes
print("🟡 Categorical Genome Demonstration")
print("="*50)

# Configuration for categorical genomes (8 positions, 5 categories each)
cat_config = mjx.CategoricalGenomeConfig(length=8, num_categories=5)
key, subkey = jr.split(key)

# Create individual genomes
cat_genome1 = mjx.CategoricalGenome.random_init(subkey, cat_config)
print(f"✓ Random categorical genome: {cat_genome1}")
print(f"  - Length: {len(cat_genome1.categories)}")
print(f"  - Number of categories: {cat_config.num_categories}")
print(f"  - Is permutation: {cat_genome1.is_permutation()}")

# Get category statistics
distribution = cat_genome1.get_category_distribution(cat_config)
count_zeros = cat_genome1.count_category(0)

print(f"  - Category distribution: {distribution}")
print(f"  - Count of category 0: {count_zeros}")

# Demonstrate categorical operations
swapped = cat_genome1.swap_positions(0, 3)
print(f"\n🔧 Categorical operations:")
print(f"  - Original positions [0,3]: {cat_genome1.categories[jnp.array([0, 3])]}")
print(f"  - After swap_positions(0,3): {swapped.categories[jnp.array([0, 3])]}")

# Create a permutation example
perm_config = mjx.CategoricalGenomeConfig(length=5, num_categories=5)
key, subkey = jr.split(key)
perm_genome = mjx.CategoricalGenome.random_init(subkey, perm_config)
permutation = perm_genome.to_permutation(perm_config)

print(f"\n🔄 Permutation conversion:")
print(f"  - Original: {perm_genome}")
print(f"  - As permutation: {permutation}")
print(f"  - Is valid permutation: {permutation.is_permutation()}")

# Distance calculation
key, subkey = jr.split(key)
cat_genome2 = mjx.CategoricalGenome.random_init(subkey, cat_config)
hamming_dist = cat_genome1.distance(cat_genome2, "hamming")
euclidean_dist = cat_genome1.distance(cat_genome2, "euclidean")

print(f"\n📏 Distances:")
print(f"  - Hamming distance: {hamming_dist}")
print(f"  - Euclidean distance: {euclidean_dist:.3f}")

print("\n" + "="*50)

🟡 Categorical Genome Demonstration
✓ Random categorical genome: <CategoricalGenome([4, 1, 4, 3, 3, 2, 1, 4], len=8)>
  - Length: 8
  - Number of categories: 5
  - Is permutation: False
  - Category distribution: [0. 2. 1. 2. 3.]
  - Count of category 0: 0

🔧 Categorical operations:
  - Original positions [0,3]: [4 3]
  - After swap_positions(0,3): [3 4]

🔄 Permutation conversion:
  - Original: <CategoricalGenome([3, 3, 0, 3, 0], len=5)>
  - As permutation: <CategoricalGenome([2, 4, 0, 1, 3], len=5)>
  - Is valid permutation: True

📏 Distances:
  - Hamming distance: 7.0
  - Euclidean distance: 5.657

✓ Random categorical genome: <CategoricalGenome([4, 1, 4, 3, 3, 2, 1, 4], len=8)>
  - Length: 8
  - Number of categories: 5
  - Is permutation: False
  - Category distribution: [0. 2. 1. 2. 3.]
  - Count of category 0: 0

🔧 Categorical operations:
  - Original positions [0,3]: [4 3]
  - After swap_positions(0,3): [3 4]

🔄 Permutation conversion:
  - Original: <CategoricalGenome([3, 3, 0, 3,

## 🟢 Linear Genomes: Symbolic Regression & Genetic Programming

Linear genomes represent **computational DAGs (Directed Acyclic Graphs)** - perfect for symbolic regression, automatic programming, and expression evolution.

### Key Features:
- **Tree-like structure**: Each instruction builds on previous results
- **Mathematical operations**: ADD, SUB, MUL, DIV with configurable arity
- **Automatic rendering**: Human-readable expression display
- **Auto-correction**: Invalid instruction arguments automatically fixed

In [43]:
# Create Linear Genomes for Genetic Programming
print("🟢 Linear Genome Demonstration")
print("="*50)

# Configuration for linear genomes (computational DAGs)
linear_config = mjx.LinearGenomeConfig(
    length=6,           # Number of instructions
    num_inputs=2,       # x_0, x_1 input variables
    num_ops=4,         # ADD, SUB, MUL, DIV operations
    max_arity=2        # Binary operations
)

key, subkey = jr.split(key)

# Create a linear genome
linear_genome = mjx.LinearGenome.random_init(subkey, linear_config)
print(f"✓ Random linear genome: {linear_genome}")
print(f"  - Length: {len(linear_genome.ops)}")
print(f"  - Operations shape: {linear_genome.ops.shape}")
print(f"  - Arguments shape: {linear_genome.args.shape}")

# Display the genome as readable expressions
print(f"\n📋 Genome as mathematical expressions:")
print(linear_genome.render(config=linear_config))

# Create linear genome population for evaluation
key, subkey = jr.split(key)
linear_population = mjx.LinearPopulation.init_random(subkey, linear_config, size=20)
print(f"\n✓ Created linear population: {len(linear_population)} genomes")

# Demonstrate evaluation with LinearGP
print(f"\n🧮 Linear GP Evaluation:")
# Create some sample data for evaluation
key, subkey = jr.split(key)
X = jr.normal(subkey, (10, 2))  # 10 samples, 2 features
y = X[:, 0]**2 + X[:, 1] - 1.0   # Target: x^2 + y - 1

# Setup evaluator with regression data
linear_evaluator = mjx.LinearGPEvaluator(config=linear_config)
data = (X, y)  # Package X and y as tuple for the evaluator

# Evaluate each genome and get the best instruction fitness from each
linear_fitness_scores = []
for genome in linear_population:
    instruction_fitnesses = linear_evaluator.evaluate(genome, data)
    best_instruction_fitness = linear_evaluator.get_best_instruction_fitness(instruction_fitnesses)
    linear_fitness_scores.append(float(best_instruction_fitness))

best_linear_fitness = max(linear_fitness_scores)
best_linear_idx = linear_fitness_scores.index(best_linear_fitness)
best_linear_genome = linear_population[best_linear_idx]

print(f"  - Best fitness: {best_linear_fitness:.6f}")
print(f"  - Target function: x^2 + y - 1")
print(f"\n🏆 Best evolved expression:")
print(best_linear_genome.render(config=linear_config))

print("\n" + "="*50)

🟢 Linear Genome Demonstration
✓ Random linear genome: <LinearGenome(L=6)>
  - Length: 6
  - Operations shape: (6,)
  - Arguments shape: (6, 2)

📋 Genome as mathematical expressions:
Row  | Expression                     | Raw
--------------------------------------------------
0    | v_0 = OP_1(x_0, x_1)           | [0 1]
1    | v_1 = OP_0(x_1, v_0)           | [1 2]
2    | v_2 = OP_2(x_1, v_0)           | [1 2]
3    | v_3 = OP_3(x_0, v_2)           | [0 4]
4    | v_4 = OP_1(v_3, x_1)           | [5 1]
5    | v_5 = OP_3(x_0, v_2)           | [0 4]

✓ Created linear population: 20 genomes

🧮 Linear GP Evaluation:


  - Best fitness: -1.587147
  - Target function: x^2 + y - 1

🏆 Best evolved expression:
Row  | Expression                     | Raw
--------------------------------------------------
0    | v_0 = OP_0(x_0, x_1)           | [0 1]
1    | v_1 = OP_1(x_1, v_0)           | [1 2]
2    | v_2 = OP_3(x_1, v_1)           | [1 3]
3    | v_3 = OP_3(v_0, x_0)           | [2 0]
4    | v_4 = OP_0(x_1, v_3)           | [1 5]
5    | v_5 = OP_2(x_1, x_1)           | [1 1]



## ⚡ Population Operations & JAX Performance

MalthusJAX populations provide **vectorized operations** for efficient manipulation of large genome collections. All operations are JAX-accelerated and JIT-compilable.

### Key Features:
- **Vectorized operations**: Process thousands of genomes simultaneously
- **Distance matrices**: Compute all pairwise distances efficiently  
- **Slicing and indexing**: List-like access to individual genomes
- **JIT compilation**: GPU acceleration for maximum performance

In [44]:
# Population Operations Demo
print("⚡ Population Operations & Performance")
print("="*60)

# Create large populations for performance testing
population_size = 1000
print(f"Creating populations with {population_size} individuals...")

key, subkey1, subkey2, subkey3 = jr.split(key, 4)

# Large binary population
large_binary_config = mjx.BinaryGenomeConfig(length=50)
large_binary_pop = mjx.BinaryPopulation.init_random(subkey1, large_binary_config, size=population_size)

# Large real population  
large_real_config = mjx.RealGenomeConfig(length=20, bounds=(-10.0, 10.0))
large_real_pop = mjx.RealPopulation.init_random(subkey2, large_real_config, size=population_size)

# Large categorical population
large_cat_config = mjx.CategoricalGenomeConfig(length=30, num_categories=8)
large_cat_pop = mjx.CategoricalPopulation.init_random(subkey3, large_cat_config, size=population_size)

print(f"✓ Binary population shape: {large_binary_pop.genes.bits.shape}")
print(f"✓ Real population shape: {large_real_pop.genes.values.shape}")
print(f"✓ Categorical population shape: {large_cat_pop.genes.categories.shape}")

# Demonstrate slicing and indexing
print(f"\\n🔍 Population Access:")
print(f"  - Individual genome: {type(large_binary_pop[0])}")
print(f"  - Slice of 5: size={len(large_binary_pop[:5])}")
print(f"  - Every 100th: size={len(large_binary_pop[::100])}")

print(f"\\n📊 Population Statistics:")
binary_ones = jnp.mean(jnp.sum(large_binary_pop.genes.bits, axis=1))
real_magnitudes = jnp.mean(jnp.linalg.norm(large_real_pop.genes.values, axis=1))
cat_diversity = len(jnp.unique(large_cat_pop.genes.categories))

print(f"  - Binary: Average ones per genome: {binary_ones:.1f}")
print(f"  - Real: Average magnitude: {real_magnitudes:.3f}")
print(f"  - Categorical: Unique values: {cat_diversity}")

print("\\n" + "="*60)

⚡ Population Operations & Performance
Creating populations with 1000 individuals...
✓ Binary population shape: (1000, 50)
✓ Real population shape: (1000, 20)
✓ Categorical population shape: (1000, 30)
\n🔍 Population Access:
  - Individual genome: <class 'malthusjax.core.genome.binary_genome.BinaryGenome'>
  - Slice of 5: size=5
  - Every 100th: size=10
\n📊 Population Statistics:
  - Binary: Average ones per genome: 25.1
  - Real: Average magnitude: 25.749
  - Categorical: Unique values: 8
\n============================================================


In [45]:
# Distance Matrices and Performance Benchmarking
print("📏 Distance Matrix Computation")
print("="*60)

# Test distance matrix computation for different genome types
print("Computing distance matrices...")

# Binary distance matrix (smaller for demo)
binary_subset = large_binary_pop[:100]
start_time = time.time()
binary_distances = binary_subset.distance_matrix()
binary_time = time.time() - start_time

print(f"✓ Binary distance matrix: {binary_distances.shape} in {binary_time*1000:.1f}ms")
print(f"  - Distance range: [{jnp.min(binary_distances):.1f}, {jnp.max(binary_distances):.1f}]")
print(f"  - Average distance: {jnp.mean(binary_distances):.2f}")

# Real distance matrix
real_subset = large_real_pop[:100] 
start_time = time.time()
real_distances = real_subset.distance_matrix()
real_time = time.time() - start_time

print(f"✓ Real distance matrix: {real_distances.shape} in {real_time*1000:.1f}ms")
print(f"  - Distance range: [{jnp.min(real_distances):.2f}, {jnp.max(real_distances):.2f}]")
print(f"  - Average distance: {jnp.mean(real_distances):.2f}")

# Performance comparison: JIT vs non-JIT
print(f"\\n🚀 JIT Compilation Performance:")

# Create evaluator for benchmarking
sphere_evaluator = mjx.SphereEvaluator(mjx.SphereConfig(minimize=True))

# Non-JIT evaluation
start_time = time.time()
fitness_normal = sphere_evaluator.evaluate_batch(large_real_pop)
normal_time = time.time() - start_time

# JIT evaluation (with warm-up)
jit_fitness_fn = jax.jit(sphere_evaluator.get_tensor_fitness_function())
_ = jit_fitness_fn(large_real_pop.genes.values[:10])  # Warm-up

start_time = time.time()
fitness_jit = jit_fitness_fn(large_real_pop.genes.values).tolist()
jit_time = time.time() - start_time

speedup = normal_time / jit_time if jit_time > 0 else float('inf')

print(f"  - Normal evaluation: {normal_time*1000:.1f}ms")
print(f"  - JIT evaluation: {jit_time*1000:.1f}ms")
print(f"  - Speedup: {speedup:.1f}x faster")
print(f"  - Results match: {jnp.allclose(jnp.array(fitness_normal), jnp.array(fitness_jit))}")

print("\\n" + "="*60)

📏 Distance Matrix Computation
Computing distance matrices...
✓ Binary distance matrix: (100, 100) in 3.3ms
  - Distance range: [0.0, 36.0]
  - Average distance: 24.72
✓ Real distance matrix: (100, 100) in 8.0ms
  - Distance range: [0.00, 20.00]
  - Average distance: 19.46
\n🚀 JIT Compilation Performance:
  - Normal evaluation: 4.6ms
  - JIT evaluation: 17.3ms
  - Speedup: 0.3x faster
  - Results match: True
\n============================================================


## 🎯 Summary & Next Steps

Congratulations! You've explored the **Level 1 foundations** of MalthusJAX. Here's what we've covered:

### ✅ Genome Types Mastered
- **🔴 Binary Genomes**: Bit strings for combinatorial optimization (OneMax, Knapsack)
- **🔵 Real Genomes**: Continuous vectors for function optimization (Sphere, Griewank) 
- **🟡 Categorical Genomes**: Discrete choices for routing and classification problems
- **🟢 Linear Genomes**: Computational DAGs for symbolic regression and GP

### ✅ Key Capabilities Demonstrated
- **Population Management**: Vectorized operations on thousands of genomes
- **Fitness Evaluation**: Efficient batch processing with automatic vectorization
- **JAX Integration**: JIT compilation for GPU acceleration
- **Distance Metrics**: Multiple similarity measures for population diversity
- **Auto-correction**: Built-in validation and constraint handling

### 🚀 Ready for Level 2!
You're now ready to explore **Level 2: Genetic Operators** including:
- **Selection**: Tournament, roulette, and elite selection strategies
- **Crossover**: Recombination operators for different genome types
- **Mutation**: Variation operators with configurable rates
- **Evolution Loops**: Complete evolutionary algorithm implementations

### 📚 Additional Resources
- **Level 2 Demo**: `examples/Level_2_Demo.ipynb` (genetic operators)
- **Level 3 Demo**: `examples/Level_3_Demo.ipynb` (evolution engines)
- **API Documentation**: Complete reference in `docs/`
- **Research Examples**: Advanced applications in `examples/`